In [1]:
from api.services.planning_optimizer.tests.data_mocker import mock_back_data
from api.services.planning_optimizer.solver import solver_planning_optimizer
from api.controllers.planning_optimizer import FrontEndPlanningOptimizerRequestContent

In [2]:
from datetime import datetime

front_end_request = FrontEndPlanningOptimizerRequestContent(backend_url="url.com",
                 backend_access_token="token",
                 date_start=datetime.fromisoformat("2023-01-18T21:01:55.124529"),
                 date_end=datetime.fromisoformat("2023-01-21T21:01:55.124529"),
                 key_project_prioritys_projets={1:1},
                 parts_max_length=1,
                 min_duration_section=0.25)

In [7]:
imperatifs, horaires, taches, utilisateurs_avec_taches_sans_horaires = mock_back_data(
            date_start=front_end_request.date_start,
            date_end=front_end_request.date_end,
            avg_n_tasks=10,
            avg_n_users=2
        )

In [8]:
list(imperatifs.values())[-1]

,evt_spkevenement,evt_sfkprojet,lgl_sfkligneparent,evt_xdate_debut,evt_xdate_fin
0,5997,3834,5754,2023-01-19 10:00:00,2023-01-19 10:20:00
1,3799,3814,5754,2023-01-19 17:25:00,2023-01-19 18:15:00
2,9290,7305,5754,2023-01-20 07:40:00,2023-01-20 08:15:00
3,8447,6604,5754,2023-01-20 10:45:00,2023-01-20 11:30:00
4,6584,5406,5754,2023-01-20 12:25:00,2023-01-20 13:15:00
5,7223,5740,5754,2023-01-20 13:35:00,2023-01-20 14:00:00


In [14]:
from pandas import DataFrame
a = DataFrame(a["events"])
a

,evt_spkevenement,evt_sfkprojet,KEY_PROJECT_PRIORITY,START,END
0,6999,6974,0,2023-01-19T10:30:00,2023-01-19T11:00:00
1,3391,6974,0,2023-01-19T11:00:00,2023-01-19T12:00:00
2,3391,6974,0,2023-01-19T12:00:00,2023-01-19T13:00:00
3,3229,3599,1,2023-01-19T13:00:00,2023-01-19T14:00:00
4,3229,3599,1,2023-01-19T14:00:00,2023-01-19T14:45:00
5,9920,3599,1,2023-01-20T08:15:00,2023-01-20T09:15:00
6,9920,3599,1,2023-01-20T09:15:00,2023-01-20T09:30:00
7,4340,2652,2,2023-01-20T09:30:00,2023-01-20T09:45:00


In [ ]:
make_timeline

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.collections import PolyCollection

from api.services.task_assigner.lib_task_assigner.tools.id_remapping import flatten_list
from api.string_keys import *

KEY_AVAILABILITIES = "AVAILABILITIES"
KEY_WORK = "WORK"

def make_timeline(availabilities: pd.DataFrame,
                  events: pd.DataFrame,
                  imperatifs: pd.DataFrame):
    """
    Dessine la timeline des événements planifiées, des disponibilités de travail et des impératifs.
    """
    data_availabilities = []

    for i, row in availabilities.iterrows():
        data_availabilities.append(
            (row[KEY_TIMESTAMP_DEBUT].to_pydatetime(),
             row[KEY_TIMESTAMP_FIN].to_pydatetime(),
             KEY_AVAILABILITIES
             ))

    data_imperatifs = []

    for i, row in imperatifs.iterrows():
        data_imperatifs.append(
            (row[KEY_TIMESTAMP_DEBUT].to_pydatetime(),
             row[KEY_TIMESTAMP_FIN].to_pydatetime(),
             MY_KEY_IMPERATIFS
             ))

    data_work = []

    for i, row in events.iterrows():
        data_work.append(
            (row[KEY_TIMESTAMP_DEBUT].to_pydatetime(),
             row[KEY_TIMESTAMP_FIN].to_pydatetime(),
             KEY_WORK
             ))

    data = data_work + data_imperatifs + data_availabilities

    cats = {MY_KEY_IMPERATIFS: 1, KEY_AVAILABILITIES: 2, KEY_WORK: 3}
    colormapping = {KEY_AVAILABILITIES: "C0", MY_KEY_IMPERATIFS: "C1", KEY_WORK: "C2"}

    verts = []
    colors = []
    for d in data:
        v = [(mdates.date2num(d[0]), cats[d[2]] - .4),
             (mdates.date2num(d[0]), cats[d[2]] + .4),
             (mdates.date2num(d[1]), cats[d[2]] + .4),
             (mdates.date2num(d[1]), cats[d[2]] - .4),
             (mdates.date2num(d[0]), cats[d[2]] - .4)]
        verts.append(v)
        colors.append(colormapping[d[2]])

    bars = PolyCollection(verts, facecolors=colors)

    fig, ax = plt.subplots(figsize=(14, 3))
    ax.add_collection(bars)
    ax.autoscale()
    loc = mdates.MinuteLocator(byminute=[0, 15, 30, 45])
    ax.xaxis.set_major_locator(loc)
    ax.xaxis.set_major_formatter(mdates.AutoDateFormatter(loc))

    ax.set_yticks([1, 2, 3])
    ax.set_yticklabels([MY_KEY_IMPERATIFS, KEY_AVAILABILITIES, KEY_WORK])

    fig.tight_layout()
    return fig

In [9]:
list(horaires.values())[-1]

,eeh_sfkperiode,eeh_xheuredebut,eeh_xheurefin
0,0,06:30,10:30
1,0,11:30,15:30
2,1,06:30,10:30
3,1,11:30,15:30
4,2,06:30,10:30
5,2,11:30,15:30
6,3,06:30,10:30
7,3,10:30,15:30
8,4,07:00,10:00


In [10]:
list(taches.values())[-1]

,evt_dduree,evt_spkevenement,lgl_sfkligneparent,evt_sfkprojet,KEY_PROJECT_PRIORITY
0,1.75,3229,5754,3599,1
1,2.00,3391,5754,6974,0
2,0.25,4340,5754,2652,2
3,0.50,6999,5754,6974,0
4,1.25,9920,5754,3599,1


In [11]:
optimized_planning = solver_planning_optimizer(
        imperatifs=imperatifs,
        working_times=horaires,
        taches=taches,
        date_start=front_end_request.date_start,
        date_end=front_end_request.date_end,
        parts_max_length=front_end_request.parts_max_length,
        min_duration_section=front_end_request.min_duration_section,
    )

In [12]:
a = list(optimized_planning["solution"].values())[-1].serialize()

In [10]:
import pandas as pd
a["START"] = pd.to_datetime(a["START"])